In [1]:
import json
from pprint import pprint
from tqdm import tqdm
from IPython.core.debugger import set_trace
import re
import copy
import os

In [3]:
ace05_ee_path = "../ori_data/ace2005/"
save_dir = "../ori_data/ace05_lu"
train_data_lu = json.load(open(ace05_ee_path + "train_data.json", "r", encoding = "utf-8"))
dev_data_lu = json.load(open(ace05_ee_path + "dev_data.json", "r", encoding = "utf-8"))
test_data_lu = json.load(open(ace05_ee_path + "test_data.json", "r", encoding = "utf-8"))

In [8]:
# from lu
train_data, dev_data, test_data = train_data_lu, dev_data_lu, test_data_lu

In [9]:
# from hmeae
# save_dir = "../ori_data/ace05_hmeae"
# hmeae_split_path = "hmeae_github_published_split.json"
# total_data_lu = {**train_data_lu, **dev_data_lu, **test_data_lu}
# total_data = {}
# for key, doc in total_data_lu.items():
#     total_data[key.split("/")[-1]] = doc

# hmeae_split = json.load(open(ace05_ee_path + hmeae_split_path, "r", encoding = "utf-8"))
# train_data, dev_data, test_data = {}, {}, {}
# for filename in hmeae_split["train"]:
#     train_data[filename] = total_data[filename]
# for filename in hmeae_split["dev"]:
#     dev_data[filename] = total_data[filename]
# for filename in hmeae_split["test"]:
#     test_data[filename] = total_data[filename]

In [10]:
tokenize = lambda text: text.split(" ")
def get_tok2char_span_map(text):
    tokens = tokenize(text)
    tok2char_span = []
    char_num = 0
    for tok in tokens:
        tok2char_span.append((char_num, char_num + len(tok)))
        char_num += len(tok) + 1 # +1: whitespace
    return tok2char_span

In [11]:
def parse_data(data):
    example_list = []
    for k, article in tqdm(data.items()):
        bio_tags = [lab.split("_")[0] for lab in article["event_label_list"]]
        bio_str = "".join(bio_tags)
        text = " ".join(article["word_list"])
        
        # find all events
        event_list = []
        for m in re.finditer("BI*", bio_str):
            trigger_char_span = [m.span()[0], m.span()[1]]
            trigger_type = article["event_label_list"][trigger_char_span[0]].split("_")[1]
            even_word_idx_list = [m.span()[0], m.span()[1]]
            trigger_start = trigger_char_span[0]
            arguments = article["event_argument_list"][trigger_start]
            arguments_new = []
            for arg in arguments:
                arg_start = trigger_start + arg["start"]
                arg_end = trigger_start + arg["end"]
                even_word_idx_list.append(arg_start)
                even_word_idx_list.append(arg_end)
                arg_text = " ".join(article["word_list"][arg_start:arg_end])
    #             assert arg_text == arg["text"].lower()
                arguments_new.append({
                    "text": arg_text,
                    "char_span": [arg_start, arg_end],
                    "type": arg["type"],
                })
            even_word_idx_list = sorted(even_word_idx_list)
            event_list.append({
                "event_span": [even_word_idx_list[0], even_word_idx_list[-1]],
                "trigger": " ".join(article["word_list"][trigger_char_span[0]:trigger_char_span[1]]),
                "trigger_type": trigger_type,
                "trigger_char_span": trigger_char_span,
                "argument_list": arguments_new,
            })

        for sent_idx, sen_ext in enumerate(article["sentence_extents"]):
            sent = " ".join(article["word_list"][sen_ext[0]:sen_ext[1]])
            tok2char_span = get_tok2char_span_map(sent)
            
            event_list_sent = []
            for event_ in event_list:
                event = copy.copy(event_)
                if event["event_span"][0] >=  sen_ext[0] and event["event_span"][1] <= sen_ext[1]:
                    for arg in event["argument_list"]:
                        arg["char_span"] = [arg["char_span"][0] - sen_ext[0], arg["char_span"][1] - sen_ext[0]]
                        char_span_list = tok2char_span[arg["char_span"][0]:arg["char_span"][1]]
                        arg["char_span"] = [char_span_list[0][0], char_span_list[-1][1]]
                        assert sent[arg["char_span"][0]:arg["char_span"][1]] == arg["text"]
                      
                    del event["event_span"]
#                     event["event_span"] = [event["event_span"][0] - sen_ext[0], event["event_span"][1] - sen_ext[0]]
#                     char_span_list = tok2char_span[event["event_span"][0]:event["event_span"][1]]
#                     event["event_span"] = [char_span_list[0][0], char_span_list[-1][1]]
                    
                    event["trigger_char_span"] = [event["trigger_char_span"][0] - sen_ext[0], event["trigger_char_span"][1] - sen_ext[0]]
                    char_span_list = tok2char_span[event["trigger_char_span"][0]:event["trigger_char_span"][1]]
                    event["trigger_char_span"] = [char_span_list[0][0], char_span_list[-1][1]]
                    assert sent[event["trigger_char_span"][0]:event["trigger_char_span"][1]] == event["trigger"]
                    
                    event_list_sent.append(event)
            example_list.append({
                "id": "{}_{}".format(article["docid"], sent_idx),
                "text": sent,
                "event_list": event_list_sent,
            })      
    return example_list

In [12]:
train_data_ = parse_data(train_data)
dev_data_ = parse_data(dev_data)
test_data_ = parse_data(test_data)

100%|██████████| 40/40 [00:00<00:00, 1376.94it/s]


In [13]:
# transform to tplinker style
def trans_data(data):
    for sent in data:
        entity_list, rel_list = [], []
        for event in sent["event_list"]:
            entity_list.append({
                "text": event["trigger"],
                "type": "{}_{}".format("Trigger", event["trigger_type"]),
                "char_span": event["trigger_char_span"],
            })
            for arg in event["argument_list"]:
                entity_list.append({
                    "text": arg["text"],
                    "type": "{}_{}".format("Argument", arg["type"]),
                    "char_span": arg["char_span"],
                })
                rel_list.append({
                    "subject": arg["text"],
                    "subj_char_span": arg["char_span"],
                    "object": event["trigger"],
                    "obj_char_span": event["trigger_char_span"],
                    "predicate": "{}_{}".format(arg["type"], event["trigger_type"]),
                })
        sent["entity_list"] = entity_list
        sent["relation_list"] = rel_list
    return data

In [14]:
normal_train_data = trans_data(train_data_)
normal_dev_data = trans_data(dev_data_)
normal_test_data = trans_data(test_data_)

In [15]:
# save 
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
json.dump(normal_train_data, open(save_dir + "/train_data.json", "w", encoding = "utf-8"), ensure_ascii = False)
json.dump(normal_dev_data, open(save_dir + "/valid_data.json", "w", encoding = "utf-8"), ensure_ascii = False)
json.dump(normal_test_data, open(save_dir + "/test_data.json", "w", encoding = "utf-8"), ensure_ascii = False)

In [16]:
count_neg = 0
for example in normal_train_data + normal_dev_data + normal_test_data:
    if len(example["relation_list"]) == 0 and len(example["entity_list"]) == 0:
        count_neg += 1
print(count_neg/len(normal_train_data + normal_dev_data + normal_test_data))

0.7362480451485687
